# Fichier Google Colab pour faire tourner le projet

In [ ]:
# Importation des fichiers de Google Drive avec le jeu de données et le code
from google.colab import drive
drive.mount('/content/drive')

# 0. Importation des fichiers et Bibliothèques

In [ ]:
# Instalation de Tensorflow_io
# Risque de problème de version

!pip install tensorflow_io

# si ça ne marche pas il faut exécuté les commandes suivantes :
#!pip uninstall tensorflow-io
#!pip install tensorflow-io==0.25.0

In [ ]:
# Extraction des fichiers zip
# Extraction des fichiers codes
from zipfile import ZipFile

file = "drive/MyDrive/datascientest_project.zip"
with ZipFile(file, 'r') as zip:
    zip.extractall()

In [ ]:
# Extraction des fichiers Model si déjà existant
file = "drive/MyDrive/Model.zip"
with ZipFile(file, 'r') as zip:
    zip.extractall()

In [ ]:
# Extraction des fichiers history pour récupérer l'historique si déjà existant
file = "drive/MyDrive/history.zip"
with ZipFile(file, 'r') as zip:
    zip.extractall()

In [ ]:
# Importation des bibliothèques utilisées

################   base
import numpy as np
import pandas as pd

################   gestion du temps et des fichiers
from time import time     # Importer la bibliothèque time et calcul du temps au début de l'exécution (t0)

from datetime import datetime

import os         # Pour lire les fichier


################   gestion audio
import librosa                # pour analyse des audio
import librosa.display


################   Affichage et image
import IPython.display as ipd  # pour lecture audio

import matplotlib.pyplot as plt        # Pour les graphique et figures
from matplotlib import cm
%matplotlib inline


###############  tensorflow pour modèle et formatage entrée
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras
from tensorflow.keras import layers

# 1. Importation module perso du modèle

In [ ]:
import Module_reco_voc as prepro
import Module_reco_voc_2 as mod

# 2. Partie preprocess transcription

In [ ]:
t0 = time()

df = prepro.Recuperation_data_txt()

t1 = time() - t0
print("Réalisé en {} secondes".format(round(t1,3)))

# 3. Partie preprocess audio

In [ ]:
# Récupération de la durée de l'audio et stockage dans le dataframe
t0 = time()

prepro.Preprocess_simple()

t1 = time() - t0
print("Réalisé en {} secondes".format(round(t1,3)))

# 4. Partie Split des data en jeux d'entraînement, de validation et de test

In [ ]:
# Séparation du dataframe en jeu de test, de validation et d'entraînement
t0 = time()

prepro.split_dataframe()

t1 = time() - t0
print("Réalisé en {} secondes".format(round(t1,3)))

# 5. Préparation du dataset

In [ ]:
# Récupération du dataframe splité
df_train, df_val, df_test = prepro.Lecture_data_split()

In [ ]:
# Définition de la fonction de maping du texte en nombre

# Liste des caractères acceptés
caracteres = [x for x in "abcdefghijklmnopqrstuvwxyz' "]

# Mapping des caractères en chiffres (int)
char_to_num = keras.layers.StringLookup(vocabulary=caracteres, oov_token="")

# Mapping (retour) des chiffres à des caractères
num_to_char = keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True)

In [ ]:
# Fonction servant au préprocess final des données pour passer d'un dataframe à un dataset de tensorflow
#
# Il y a deux parties :
#                       - La première : transformation de l'audio en spectrogramme
#                       - La seconde : Mapping de la transcription en nombre

#================== 1. Transformation de l'audio en spectrogramme ==================#
# Paramètre de la transformé courte de fournier utilisé pour l'optention du spectrogramme
# Taille de la fenêtre en échantillons audio
frame_length = 512

# Pas d'échantillonnage entre le départ de deux fenêtre
frame_step = 128

# Nombre d'échantillon pour une durée fixé sur laquelle on applique la FFT
fft_length = 512

# Pour optimiser le temps de calcul il est recommandé d'utiliser un n_fft = 2^n (puissance de 2)
# (2/4/8/16/32/64/128/256/512/1024/2048/4096/...)
# Dans notre cas (pour le traitement de la voix) il est recommander d'utiliser 512 
# cela correspond à une période d'échantillonnage de 32 milisecondes


#================== 2. Mapping de la transcription en nombre ==================#
def Recup_spectrogramme_transcription(fichier_audio, transcription):
    #  ==== Récupération du spectrogramme ==== #
    # Lecture du fichier audio
    fichier = tf.io.read_file(fichier_audio)

    # Decodage du fichier audio .flac
    audio = tfio.audio.decode_flac(fichier, dtype = tf.int16)
    audio = tf.squeeze(audio, axis=-1)

    # Passage de l'audio en float32
    audio = tf.cast(audio, tf.float32)    # pas utile dans notre cas

    # Récupération spectrogramme
    spectrogram = tf.signal.stft(audio,
                                 frame_length = frame_length,
                                 frame_step = frame_step,
                                 fft_length = fft_length
                                )

    # On ne conserve que la racine carré de la valeur absolue du nombre du complexe
    spectrogram = tf.abs(spectrogram)              # valeur absolue
    spectrogram = tf.math.pow(spectrogram, 0.5)    # racine carrée

    # normalisation du spectrogramme
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)


    # ==== Mapping de la transcription ==== #
    # Passage de la transcription en minuscules
    transcription = tf.strings.lower(transcription)

    # Séparationt de la transcription
    transcription = tf.strings.unicode_split(transcription, input_encoding="UTF-8")

    # Map les caractères de la transcription en nombres
    transcription = char_to_num(transcription)

    return spectrogram, transcription

In [ ]:
# ==== Génération des dataset à partir des dataframes ==== #

# taille des batch
batch_size = 28

# Création du dataset d'entrainnement
train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_train["chemin"]), list(df_train["transcription"]))
)
train_dataset = (
    train_dataset.map(Recup_spectrogramme_transcription, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    #.prefetch(buffer_size=tf.data.AUTOTUNE)
)

# Création du dataset de validation
validation_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_val["chemin"]), list(df_val["transcription"]))
)
validation_dataset = (
    validation_dataset.map(Recup_spectrogramme_transcription, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

# 6. Partie construction du modèle

In [ ]:
# Création d'un dossier (si inexistant) pour les sauvegarde du modèle au cours de l'entraînement
checkpoint_doss = "./save_model_run"

if not os.path.exists(checkpoint_doss):
    os.makedirs(checkpoint_doss)

In [ ]:
# === Restoration du modèle ou création d'un nouveau modèle === #
model = mod.Fabriquer_ou_restorer_model(output_dim = char_to_num.vocabulary_size(), checkpoint_doss = checkpoint_doss)

In [ ]:
# Création du callbacks ModelCheckpoint pour sauvegarder le modèle tous les x batch
ModelCheckpoint = keras.callbacks.ModelCheckpoint(filepath=checkpoint_doss + "/Model.hdf5",
                                                  verbose=1,
                                                  save_freq=100)

In [ ]:
# Création du callback EarlyStopping pour stopper l'entraînement du modèle
# si la perte sur le dataset de validation n'a pas évolué pendant 3 épochs à partir de la 25eme épochs
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, start_from_epoch = 30)

In [ ]:
# Définition du callback ReduceLROnPlateau pour réduire le learning rate
# si la perte sur le dataset de validation n'a pas évolué pendant 3 épochs avec une valeur seuil de 1e-7
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=3, min_lr=1e-7)

In [ ]:
# Définition de la liste des callbacks utilisés
callbacks = [ModelCheckpoint,
             early_stopping,
             reduce_lr
            ]

# 7. Partie Entraînement du modèle

In [ ]:
# Entrainnementr du modèle
History = model.fit(train_dataset,
                    epochs = 20,
                    callbacks = callbacks,
                    validation_data = validation_dataset)

files.download("./save_model_run/Model.hdf5")

mod.sauvegarde_history_file(History)

files.download("history.csv")

In [ ]:
# Importation de la bibliothèque pour télécharger les résultats
from google.colab import files

In [ ]:
# Téléchargement du modèle post-entrainement 
files.download("./save_model_run/Model.hdf5")

In [ ]:
# Récupération de l'historique de l'entraînement
mod.sauvegarde_history_file(History)

In [ ]:
# Téléchargement de l'historique du modèle post-entrainement 
files.download("history.csv")

# 8. Partie Analyse du modèle et Prediction

In [ ]:
mod.analyze_metrics(History)

In [ ]:
history = pd.read_csv('history.csv')
history

In [ ]:
fig = plt.figure(figsize = (12,4))

ax1 = fig.add_subplot(121)
ax1.plot(history['epochs'], history['loss'], label='Loss')
ax1.plot(history['epochs'], history['val_loss'], label='Validation Loss')

ax1.set_xlabel('Epoch')
ax1.set_ylabel('Valeur loss')
ax1.legend()
plt.title('Valeur loss par epoch lors de l\'entraînement')

ax2 = fig.add_subplot(122)
ax2.plot(history['epochs'], history['CER_metric'], label='CER_metric')
ax2.plot(history['epochs'], history['val_CER_metric'], label='Validation CER_metric')

ax2.set_xlabel('Epoch')
ax2.set_ylabel('Valeur metric')
ax2.legend()

plt.title('Métriques par epoch lors de l\'entraînement')
plt.show()

In [ ]:
# Création du dataset de test final à partir de df_test
test_dataset = tf.data.Dataset.from_tensor_slices(
    (list(df_test["chemin"]), list(df_test["transcription"]))
)
test_dataset = (
    test_dataset.map(Recup_spectrogramme_transcription, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
)

In [ ]:
# Fonction pour décoder la prédiction en texte
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]

    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]

    output_text = []
    for result in results:
        result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
        output_text.append(result)
    return output_text

In [ ]:
# Comparaisson cible / prédiction sur le jeu de donnée test
predictions = []
targets = []
for batch in test_dataset.take(1):
    X, y = batch
    batch_predictions = model.predict(X)
    batch_predictions = decode_batch_predictions(batch_predictions)
    predictions.extend(batch_predictions)
    for label in y:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        targets.append(label)

for i in np.random.randint(0, len(predictions), 5):
    print(f"Target    : {targets[i]}")
    print(f"Prediction: {predictions[i]}")
    print("-" * 100)

In [ ]:
# Comparaisson cible / prédiction sur le jeu de donnée train
predictions = []
targets = []
for batch in train_dataset.take(1):
    X, y = batch
    batch_predictions = model.predict(X)
    batch_predictions = decode_batch_predictions(batch_predictions)
    predictions.extend(batch_predictions)
    for label in y:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        targets.append(label)

for i in np.random.randint(0, len(predictions), 5):
    print(f"Target    : {targets[i]}")
    print(f"Prediction: {predictions[i]}")
    print("-" * 100)